# Milestone 5: Morphology helper

Setup

In [ ]:
import getpass
import logging
from pathlib import Path

from bluepysnap.api import Api

logging.basicConfig(level=logging.WARNING)

ORGANIZATION = "bbp"
PROJECT = "thalamus"

Paste the token from https://bbp.epfl.ch/nexus/web/

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
nexus_config = "../../../examples/configurations/api/forge.yml"
api = Api(nexus_config=nexus_config, bucket=f"{ORGANIZATION}/{PROJECT}", token=TOKEN, debug=False)

### From a KG URL retrieve a Morphology Collection

In [ ]:
morphology_release_id = "https://bbp.epfl.ch/neurosciencegraph/data/20191208"
morphology_release = api.get_entity_by_id(morphology_release_id)
print(morphology_release)

### Get information related to a Morphology Collection

In [ ]:
print("Description:", morphology_release.description)
print("Creator:", morphology_release.createdBy)
print("Creation date:", morphology_release.createdAt)
print("Project:", morphology_release.project)
print("Species:", morphology_release.species.label)
print("Brain region:", morphology_release.brainLocation.brainRegion.label)

### Retrieve and access a nested resource

In [ ]:
print(morphology_release.morphologyIndex.distribution.url)

### Show the wrapped Nexus resource as json

In [ ]:
api.as_json(morphology_release)

### Show the wrapped Nexus resource as dataframe

In [ ]:
api.as_dataframe([morphology_release]).T

### Instantiate a MorphDB object from a Morphology Collection and a unique identifier

In [ ]:
# this can be slow
morphology_release.instance

In [ ]:
df = morphology_release.instance.df
df

In [ ]:
df["mtype"].value_counts()

### Get the morphologies from the morphology release entity

#### From a morphology release df
This method requires access to the internal df that can depend on the library used to open the morphology release, so it's not recommended unless needed for performance or flexibility reasons.

Note: `api.morphology` is a namespace used for convenience to group methods related to morphologies.

In [ ]:
df = morphology_release.instance.df
morphologies = api.morphology.get_morphologies_from_df(df.iloc[0:2])
morphologies

In [ ]:
morphologies = api.morphology.get_morphologies_from_df(df[df['mtype'] == 'VPL_IN'])
len(morphologies)

#### From a morphology release entity
This method is slightly more generic: it doesn't expose directly the internal dataframe, but it allows to specify a query in the format required by `pandas` without having access to the dataframe.

Alternatively, if we don't need to be able to specify a generic query, we could expose a different method that can be more specific, filtering on a limited set of columns.

In [ ]:
morphologies = api.morphology.get_morphologies_from_morphology_release(
    morphology_release, 'mtype == ["VPL_IN", "Rt_RC"]'
)
len(morphologies)

### Plot a morphology

In [ ]:
morphology = morphologies[0]
print(type(morphology.instance))
print(morphology.instance.__class__)

In [ ]:
%matplotlib inline
from neurom import viewer
fig, ax = viewer.draw(morphology.instance, mode="2d")

### ~~Get the mean diameter of the L23 cells of that morphology release~~
    Get the mean soma radius of the VPL cells of that morphology release

In [ ]:
morphologies = api.morphology.get_morphologies_from_morphology_release(
    morphology_release, 'layer == "Rt"'
)

In [ ]:
import neurom as nm
from neurom.core.population import Population
# this can be slow on the first execution
population = Population([m.instance for m in morphologies])

In [ ]:
import numpy as np
print(np.mean(nm.get('soma_radii', population)))